DISNEY DATASET CREATION

Developing program idea on 1 example movie

In [45]:
#Task1. Get info box

from bs4 import BeautifulSoup as bs
import requests

In [46]:
#Load demo webpage
r = requests.get("https://en.wikipedia.org/wiki/Pinocchio_(1940_film)")

#Convert to a beautiful soup object
soup = bs(r.content)

#Print out the HTML
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Pinocchio (1940 film) - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.cookie.match(/

In [47]:
info_box = soup.find(class_="infobox vevent")  # finding object in HTML code that has class name "infobox vevent"
info_box.prettify()

print(info_box)

<table class="infobox vevent"><tbody><tr><th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">Pinocchio</th></tr><tr><td class="infobox-image" colspan="2"><a class="image" href="/wiki/File:Pinocchio-1940-poster.jpg"><img alt="Pinocchio-1940-poster.jpg" class="thumbborder" data-file-height="383" data-file-width="260" decoding="async" height="324" src="//upload.wikimedia.org/wikipedia/en/thumb/b/ba/Pinocchio-1940-poster.jpg/220px-Pinocchio-1940-poster.jpg" srcset="//upload.wikimedia.org/wikipedia/en/b/ba/Pinocchio-1940-poster.jpg 1.5x" width="220"/></a><div class="infobox-caption">Theatrical release poster</div></td></tr><tr><th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">Directed by</th><td class="infobox-data"><b>Supervising Directors</b><style data-mw-deduplicate="TemplateStyles:r1126788409">.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding

In [48]:
info_rows = info_box.find_all("tr")   #choosing all rows containing "tr"

for row in info_rows:
    print(row.prettify())             #in the list getting all HTML lines.

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Pinocchio
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Pinocchio-1940-poster.jpg">
   <img alt="Pinocchio-1940-poster.jpg" class="thumbborder" data-file-height="383" data-file-width="260" decoding="async" height="324" src="//upload.wikimedia.org/wikipedia/en/thumb/b/ba/Pinocchio-1940-poster.jpg/220px-Pinocchio-1940-poster.jpg" srcset="//upload.wikimedia.org/wikipedia/en/b/ba/Pinocchio-1940-poster.jpg 1.5x" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="infobox-data">
  <b>
   Supervising Directors
  </b>
  <style data-mw-deduplicate="TemplateStyles:r1126788409">
   .mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-s

In [49]:
import re

# function to create data dictionary
def get_data_dictionary(info_box):
    #saving into a dictionary
    movie_info = {}

  
    info_rows = info_box.find_all("tr")   #choosing all rows containing "tr"

    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['Title'] = row.find("th").get_text(" ", strip=True)

        elif index == 1:
            continue  

        elif row.find('span', class_='bday dtstart published updated'):
            content_key = row.find("th").get_text(" ", strip=True)

            # Extract the date values and collect them in a list
            content_data = []
            for span in row.find_all('span', class_='bday dtstart published updated'):
                content_data.append(span.get_text(" ",strip=True))

            for index in range(len(content_data)):
                    item = content_data[index]
                    content_data[index] = re.sub(r'\s*\[.*?\]', '', item)
            
            movie_info[content_key] = content_data
            
            
         #checking for table data that contains multiple elemets
        elif row.find("li"):
            content_key = row.find("th").get_text(" ", strip=True)

            # Extract all table values and collect them in a list
            content_data = []
            for li in row.find_all("li"):
                content_data.append(li.get_text(" ", strip=True))
                        
                for i, element in enumerate(content_data):
                    # replacing all "\xa0"s with space
                    if "\xa0" in element:
                        content_data[i] = element.replace("\xa0", " ")    

            # finding possible refernces from HTML and removing them
            content_data = [re.sub(r'\s*\[.*?\]', '', item) for item in content_data]

            movie_info[content_key] = content_data

        # default
        else:
            content_key = row.find("th").get_text(" ", strip=True)
                
            content_data = row.find("td")
            content_data = content_data.get_text(" ", strip=True)

            # finding possible refernces from HTML
            for i in range(10):
                str_item = f"[{i}]"
                if str_item in content_data:
                    content_data = content_data.replace(str_item, "")

            movie_info[content_key] = content_data

    return movie_info

In [50]:
movie = get_data_dictionary(info_box)
movie["Release dates"]

['1940-02-07', '1940-02-23']

WORKING ON WALT_DISNAY WEBPAGE

Applying method to create data dictionary before on all of the webpages

In [51]:
#Load webpage - Toy story
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#Convert to a beautiful soup object
soup =bs(r.content)

#Print out the HTML
contents = soup.prettify()
print(contents)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Walt Disney Pictures films - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-enabled vector-feature-main-menu-pinned-disabled vector-feature-limited-width-enabled vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled";(function(){var cookie=document.c

In [52]:
#getting rows that contains wikipedia URL
container = soup.select(".wikitable.sortable i")
filtered_container = [row for row in container if "/wiki" in str(row)]

filtered_container

[<i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>,
 <i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>,
 <i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>,
 <i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>,
 <i><a href="/wiki/Dumbo" title="Dumbo">Dumbo</a></i>,
 <i><a href="/wiki/Bambi" title="Bambi">Bambi</a></i>,
 <i><a href="/wiki/Saludos_Amigos" title="Saludos Amigos">Saludos Amigos</a></i>,
 <i><a href="/wiki/Victory_Through_Air_Power_(film)" title="Victory Through Air Power (film)">Victory Through Air Power</a></i>,
 <i><a href="/wiki/The_Three_Caballeros" title="The Three Caballeros">The Three Caballeros</a></i>,
 <i><a href="/wiki/Make_Mine_Music" title="Make Mine Music">Make Mine Music</a></i>,
 <i><a href="/wiki/Song_of_the_Sou

In [53]:
# getting URL of webpages
def get_movie_url(row):

    head = 'https://en.wikipedia.org/'
    url = row.a['href']

    url = head + url

    return url

# using URL getting InfoBox of webpage
def get_info_box(url):
    # Load webpage
    r = requests.get(url)

    # Convert to a beautiful soup object
    soup = bs(r.content)

    info_box = soup.find(class_="infobox vevent")

    return info_box

In [54]:
# list to store whole data from all webpages
movie_data = []

for row in filtered_container:
    #using try-except block to avoid possible errors (if possible solve them)
    try:
        url = get_movie_url(row)
        info_box = get_info_box(url)
        dictionary = get_data_dictionary(info_box)

        movie_data.append(dictionary)
    except Exception as e:
        print(row.get_text())
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'get_text'
The Sign of Zorro
'NoneType' object has no attribute 'get_text'
Mighty Ducks the Movie: The First Face-Off
'NoneType' object has no attribute 'get_text'
Spirited Away
'NoneType' object has no attribute 'get_text'
Howl's Moving Castle
'NoneType' object has no attribute 'get_text'
The Nightmare Before Christmas 3D
'NoneType' object has no attribute 'get_text'
The Secret of the Magic Gourd
'NoneType' object has no attribute 'get_text'
Ponyo
'NoneType' object has no attribute 'get_text'
Tales from Earthsea
'NoneType' object has no attribute 'get_text'
Anaganaga O Dheerudu
'NoneType' object has no attribute 'get_text'
The Secret World of Arrietty
'NoneType' object has no attribute 'get_text'
Tini: The Movie
'NoneType' object has no attribute 'get_text'
Born in China
'NoneType' object has no attribute 'get_text'
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'get_text'
Elio
'NoneType' object has n

Save/Reload final data list in json data file form

In [55]:
import json

def save_data_json(title, data):
    with open(title, 'w', encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def load_data_json(title):
    with open(title, encoding='utf-8') as f:
        data = json.load(f)
        return data

In [56]:
save_data_json("disney_movies_data.json", movie_data)

Data Cleaning

In [57]:
data = load_data_json("disney_movies_data.json")
len(data)

534

In [58]:
from datetime import datetime
import re

def convert_date_format(old_date): 
    if isinstance(old_date, datetime):
        return old_date
    
    elif isinstance(old_date, str):
        if old_date is not None:
            # checking for type YYYY-MM-DD
            pattern = r"\d{4}-\d{2}-\d{2}"

            if re.match(pattern, old_date):
                new_date = datetime.strptime(old_date, "%Y-%m-%d").date()
                return new_date

            # checking for type YYYY-MM
            pattern = r"\d{4}-\d{2}"

            if re.match(pattern, old_date):
                new_date = datetime.strptime(old_date, "%Y-%d").date()
                return new_date

            else:
                # Separate and change elements in the date
                separated_date = old_date.split()

                # To delete elements having "(", ")", and "["
                filtered_date = [elem for elem in separated_date if "(" not in elem and ")" not in elem and "[" not in elem]

                # Removing commas from elements
                filtered_date = [elem.replace(',', '') for elem in filtered_date]

                # List of months
                month = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

                # Changing name of months to corresponding number
                for i in range(len(filtered_date)):
                    if filtered_date[i] in month:
                        filtered_date[i] = str(month.index(filtered_date[i]) + 1)

                year_index = []

                # Getting indexes of years in filtered_date list
                for index, elm in enumerate(filtered_date):
                    if elm not in month:
                        if len(elm) == 4:
                            year_index.append(index)

                temp_sublist = []
                sublist = []

                # Separating filtered_date into lists that contain the actual date in the desired format
                for index, elm in enumerate(filtered_date):
                    sublist.append(elm)
                    if index in year_index:
                        temp_sublist.append(sublist)
                        sublist = []

                if sublist:
                    temp_sublist.append(sublist)

                new_date = []

                # Making format of date YYYY-MM-DD
                for elm in temp_sublist:
                    # For YYYY-MM-DD
                    if len(elm) == 3:
                        date = elm[2] + "-" + elm[0].zfill(2) + "-" + elm[1].zfill(2)
                        new_date.append(date)
                    # For YYYY-MM
                    elif len(elm) == 2:
                        date = elm[1] + "-" + elm[0].zfill(2)
                        new_date.append(date)
                    elif len(elm) == 1:
                        new_date.append(elm[0])
                    else:
                        # Handle the case where the list is empty or has more than 3 elements
                        new_date.append("N/A")

                # Final checking for the format of the date. If it contains strange things, remove it
                valid_dates = []
                for date in new_date:
                    if re.match(r"\d{4}-\d{2}-\d{2}", date) or re.match(r"\d{4}-\d{2}", date):
                        valid_dates.append(date)

                if valid_dates:
                    # checking for type YYYY-MM-DD
                    pattern = r"\d{4}-\d{2}-\d{2}"

                    if re.match(pattern, valid_dates[0]):
                        return datetime.strptime(valid_dates[0], "%Y-%m-%d").date()
                    else:
                        return datetime.strptime(valid_dates[0], "%Y-%d").date()
                    
                else:
                    return None
        else:
            return None
    else:
        if old_date is None:
            return None
        else:
            for elm in old_date:
                if elm != "N/A" and elm is not None:
                    new_date = convert_date_format(elm)
                    return new_date
                else:
                    return None

In [59]:
for movie in data:
    old_date = movie.get("Release dates")
    new_date = convert_date_format(old_date)
    movie["Release dates"] = new_date

In [60]:
print(len(data))
for movie in data:
    date = movie.get("Release dates")
    if date is not None:
        print(type(date))

534
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'date

In [61]:
import pickle

def save_data_pickle(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def load_data_pickle(file_path):
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data

In [62]:
# date type can not be saved in json file, so use pickle file
save_data_pickle(data, "disney_movies_updated.pickle")

In [63]:
# converting Running time values form "80 minutues" structure to "80" integer value
data_updated = load_data_pickle("disney_movies_updated.pickle")

def convert_RunningTime_format(run_time):
    if run_time is not None:
        run_time = run_time.split()
        return int(run_time[0])
    else:
        return "N/A"

for movie in data_updated:
    run_time = movie.get("Running time")
    time = convert_RunningTime_format(run_time)
    movie["Running time"] = time

    movie["Running time (min)"] = movie.pop("Running time")

print(len(data_updated))

534


In [64]:
# date type can not be saved in json file, so use pickle file
save_data_pickle(data_updated, "disney_movies_updated.pickle")

In [65]:
# converting format of budget
def convert_currency_format(budget):
    if budget is None:
        return "N/A"
    
    else:
        try:
            if isinstance(budget, str):

                # removing text in ( )
                budget = re.sub(r'\([^)]*\)', '', budget)
                # removing text in [ ]
                budget = re.sub(r'\[[^\]]*\]', '', budget)

                budget_seperated = budget.split()

                if len(budget_seperated) == 1:
                    amount = budget_seperated[0].replace("$", "").replace(">", "").replace("<", "")
                    budget_updated = float(amount.replace(",", ""))
                    return budget_updated
                
                if len(budget_seperated) == 2:
                    
                    if "million" in budget_seperated:
                        amount = budget_seperated[0]
                        
                        # checking for currency:
                        if "AU$" in amount:
                            budget_updated = round(float(amount.replace("AU$", "").replace(">", "")) * 750000, 2)
                            return budget_updated
                        
                        elif "A$" in amount:
                            budget_updated = round(float(amount.replace("A$", "").replace(">", "")) * 750000, 2)
                            return budget_updated
                        
                        elif "-" in amount:
                            # removing "$" sign and seperating string by "-"
                            amount = amount.replace("$", "").replace(">", "")
                            amount = amount.split("-")

                            budget_updated = (float(amount[0]) + float(amount[1])) * 500000
                            
                            return budget_updated
                        
                        elif "–" in amount:
                            # removing "$" sign and seperating string by "-"
                            amount = amount.replace("–", "-")
                            amount = amount.replace("$", "").replace(">", "")
                            amount = amount.split("-")

                            budget_updated = (float(amount[0]) + float(amount[1])) * 500000
                            
                            return budget_updated

                        else:
                            budget_updated = float(amount.replace("$", "").replace(">", "")) * 1000000
                            return budget_updated
                        
                    elif "crore" in budget_seperated:
                        amount = budget_seperated[0]
                        budget_updated = float(amount.replace("₹", "").replace(">", "")) * 130000

                        return budget_updated
                
                # for cases like there are more than 2 word in input string
                else:
                    # checking for case when there is "–" in the input
                    for elm in budget_seperated:
                        if "–" in elm:
                            amount = []
                            for index, elm in enumerate(budget_seperated):
                                # Regular expression pattern for any digit
                                pattern = r'\d'
                                match = re.search(pattern, elm)

                                # finding elm that contains any numerical values
                                if match is not None:
                                    if index < len(budget_seperated) -1:
                                        # if next element is "million", then multiplying by 1000000
                                        if budget_seperated[index+1] == "million":
                                            
                                            price = float(elm.replace("$", "").replace("–", "").replace(">", "")) * 1000000
                                            

                                            amount.append(price)

                                        else:
                                            price = float(elm.replace("$", "").replace("–", "").replace(",", "").replace(">", ""))
                                            
                                            amount.append(price)

                            budget_updated = 0

                            # finding sum of prices
                            for item in amount:
                                budget_updated += item
                            
                            # finding average of prices
                            budget_updated = budget_updated / len(amount)

                            return budget_updated

                    # checking for cases like there is "or" in the string
                    if "or" in budget_seperated:
                        # list to store numerical values
                        
                        amount = []

                        for index, elm in enumerate(budget_seperated):
                            # Regular expression pattern for any digit
                            pattern = r'\d'
                            match = re.search(pattern, elm)

                            # finding elm that contains any numerical values
                            if match is not None:
                                # if next element is "million", then multiplying by 1000000
                                if index < len(budget_seperated)-1:
                                    if budget_seperated[index+1] == "million":
                                        price = float(elm.replace("$", "").replace(">", "")) * 1000000
                                    
                                        amount.append(price)

                                # else just getting float type amount
                                else:
                                    price = elm.replace("$", "").replace(">", "")
                                    price = float(price.replace(",", ""))
                                    
                                
                                    amount.append(price)

                        budget_updated = 0

                        # to sum all items in amount list
                        for item in amount:
                            budget_updated += item

                        # find average of items
                        budget_updated = budget_updated / 2

                        return budget_updated

                    # checking for cases like there is "to" in the string
                    elif "to" in budget_seperated:
                        # list to store numerical values
                        amount = []

                        for elm in budget_seperated:
                            # Regular expression pattern for any digit
                            pattern = r'\d'
                            match = re.search(pattern, elm)

                            if match is not None:
                                amount.append(elm)
                        
                        # if there is million in the list, multiplying by 1000000
                        if "million" in budget_seperated:
                            for index, item in enumerate(amount):
                                amount[index] = float(item.replace("$", "").replace(">", "")) * 1000000


                            budget_updated = 0

                            # getting sum of amount items
                            for item in amount:
                                budget_updated += item
                            
                            # getting average of amounts in the list
                            budget_updated = budget_updated / len(amount)

                            return budget_updated
                        
                        else:
                            # removing signs
                            for index, item in enumerate(amount):
                                amount[index] = item.replace("$", "").replace(">", "")
                                amount[index] = float(amount.replace(",", ""))

                            budget_updated = 0

                            # getting sum of amount items
                            for item in amount:
                                budget_updated += item

                            # getting average of amounts in the list
                            budget_updated = budget_updated / len(amount)

                            return budget_updated
                        
                    else:
                        return "N/A"

            elif isinstance(budget, list):
                list_amount = []

                for elm in budget:
                    # removing text in ( )
                    elm = re.sub(r'\([^)]*\)', '', elm)
                    # removing text in [ ]
                    elm = re.sub(r'\[[^\]]*\]', '', elm)
                    
                    elm_seperated = elm.split()

                    # taking amount number
                    amount = elm_seperated[0]

                    amount = float(amount.replace("$", "").replace(">", "")) * 1000000

                    list_amount.append(amount)

                budget_updated = 0

                for item in list_amount:
                    budget_updated += item

                budget_updated = budget_updated / len(list_amount)

                return budget_updated

        except Exception as e:
            print(budget)
            print(e)

In [66]:
data_updated = load_data_pickle("disney_movies_updated.pickle")

for movie in data_updated :
    budget = movie.get("Budget")
    new_budget = convert_currency_format(budget)

    movie["Budget"] = new_budget

    movie["Budget ($)"] = movie.pop("Budget")

print(len(data_updated ))

$30—$35 million 
could not convert string to float: '30—35'
$30—$35 million 
could not convert string to float: '30—35'
$30—35 million 
could not convert string to float: '30—35'
₽650 million
could not convert string to float: '₽650'
₽454 million
could not convert string to float: '₽454'
~$70 million 
could not convert string to float: '~70'
$11—15 million  
could not convert string to float: '11—15'
534


In [67]:
save_data_pickle(data_updated, "disney_movies_updated.pickle")

In [68]:
data_updated = load_data_pickle("disney_movies_updated.pickle")

for movie in data_updated:
    box_office = movie.get("Box office")
    new_box_office = convert_currency_format(box_office)
    movie["Box office"] = new_box_office

₹1,968−2,200 crore 
could not convert string to float: '1,968−2,200'
['₽2.046 billion', '$28.3 million (Worldwide)', '$28.6 million (Russia/ CIS )']
could not convert string to float: '₽2.046'
['₽2.196 billion', '$27 million']
could not convert string to float: '₽2.196'


In [69]:
for movie in data_updated:
    box_office = movie.get("Box office")
    movie["Box office ($)"] = movie.pop("Box office")

In [70]:
save_data_pickle(data, "disney_movies_final_data.pickle")

In [71]:
pickle_data = load_data_pickle("disney_movies_final_data.pickle")

pickle_data[10]

{'Title': 'Song of the South',
 'Directed by': ['Live action:',
  'Harve Foster',
  'Animation:',
  'Wilfred Jackson'],
 'Screenplay by': ['Live action:',
  'Morton Grant',
  'Maurice Rapf',
  'Dalton S. Reymond',
  'Animation:',
  'Bill Peet',
  'George Stallings',
  'Ralph Wright'],
 'Based on': 'Uncle Remus by Joel Chandler Harris',
 'Produced by': 'Walt Disney',
 'Starring': ['Ruth Warrick',
  'Lucile Watson',
  'Hattie McDaniel',
  'James Baskett',
  'Bobby Driscoll',
  'Luana Patten'],
 'Cinematography': 'Gregg Toland',
 'Edited by': 'William M. Morgan',
 'Music by': ['Daniele Amfitheatrof', 'Paul J. Smith'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': datetime.date(1946, 11, 12),
 'Running time': '94 minutes ',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$2.125 million ',
 'Box office': '$65 million '}

In [74]:
import pandas as pd

df = pd.DataFrame(pickle_data)

df.head()

,Title,Directed by,Story by,Based on,Produced by,Starring,Music by,Production company,Distributed by,Release dates,...,Countries,Color process,Production companies,Layouts by,Music,Lyrics,Book,Basis,Productions,Awards
0,Snow White and the Seven Dwarfs,"[David Hand, Perce Pearce, William Cottrell, L...","[Ted Sears, Richard Creedon, Otto Englander, D...",Snow White by the Brothers Grimm,Walt Disney,"[Adriana Caselotti, Roy Atwell, Pinto Colvig, ...","[Frank Churchill, Leigh Harline, Paul Smith]",Walt Disney Productions,RKO Radio Pictures,1937-12-21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pinocchio,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...","[Ted Sears, Otto Englander, Webb Smith, Willia...",The Adventures of Pinocchio by Carlo Collodi,Walt Disney,"[Cliff Edwards, Dickie Jones, Christian Rub, W...","[Leigh Harline, Paul J. Smith]",Walt Disney Productions,RKO Radio Pictures,1940-02-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fantasia,"[Samuel Armstrong, James Algar, Bill Roberts, ...","[Joe Grant, Dick Huemer]",NaN,"[Walt Disney, Ben Sharpsteen]","[Leopold Stokowski, Deems Taylor]",See program,Walt Disney Productions,RKO Radio Pictures,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,The Reluctant Dragon,Alfred Werker (live action) Hamilton Luske (an...,NaN,NaN,Walt Disney,Robert Benchley Frances Gifford Buddy Pepper N...,Frank Churchill Larry Morey,Walt Disney Productions,RKO Radio Pictures,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dumbo,"[Ben Sharpsteen, Norman Ferguson, Wilfred Jack...","[Joe Grant, Dick Huemer]","Dumbo, the Flying Elephant by Helen Aberson Ha...",Walt Disney,"[Edward Brophy, Verna Felton, Cliff Edwards, H...",Frank Churchill Oliver Wallace,Walt Disney Productions,RKO Radio Pictures,1941-10-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df.to_csv("disney_movies_final_data.csv")